## Trabalho da disciplina Algebra e Criptografia - Implementação do RSA

#### O que é o RSA? 

O algoritmo RSA é um dos métodos de criptografia de chave pública mais conhecido e amplamente utilizado para transmissão segura de dados. Seu objetivo é proteger dados e garantir autenticidade em diversas aplicações, como comunicação segura e assinaturas digitais.

Ele foi desenvolvido em 1977 por Ron Rivest, Adi Shamir e Leonard Adleman cujas iniciais formam o nome "RSA" que atuavam como pesquisadores do Massachusetts Institute of Technology (MIT).

Tal método nos permite codificar uma mensagem utilizando o produto de dois números primos, e para decodificá-la é necessário conhecer estes primos, ou seja, decifrar a mensagem consiste em fatorar o número que é o produto dos primos usados na fase de codificação. Por isso, utilizar como chave de codificação números primos grandes faz com que a tarefa de decodificar a mensagem seja difícil e tenha alto custo computacional.


### Como funciona o RSA?

Ele utiliza um par de chaves: uma pública $(n,e)$ que é usada para criptografar e uma privada $(d)$ e é usada para descriptografar.

1. São escolhidos dois primos grandes $p$ e $q$;
2. Calculamos $ n = pq$, que será usado tanto na chave pública quanto na privada;
3. Calculamos a função totiente de Euler $\phi(n) = (p-1)(q-1)$;
4. Escolhemos um inteiro com $2<e<\phi(n)$ e $mdc(e, \phi(n)) =1$, ou seja, $e$ e $\phi(n)$ são coprimos;
5. Dada uma mensagem $m$ com $0 \leq m \leq n-1$, a mensagem codificada é $c = m^e(mod(n))$;
6. Calculamos $d$, que é o inverso multipicativo de $e \mod(\phi(n))$
7. Para codificar é necessário calcular $c^d(mod(n))$.


### Escolha dos números primos

- função $number.getPrime()$: 
A função $getPrime$ é parte da biblioteca PyCryptodome, projetada para gerar números primos de forma eficiente e segura. Ela utiliza métodos baseados em testes probabilísticos, como o teste de Miller-Rabin, para produzir números primos com o número de bits especificado, garantindo sua adequação para aplicações criptográficas.
Neste trabalho, os valores de $p$ e $q$ podem ser determinados condicionalmente. Se a opção for configurada como $True$, a função $getPrime$ é utilizada para gerar $p$ e $q$. Caso contrário ($False$), os valores de $p$ e $q$ devem ser fornecidos manualmente.

- gerador de primos da Marcelli e do Falqueto: está por vir

### Algoritmo Estendido de Euclides

O algoritmo estendido de Euclides é uma extensão do algoritmo de Euclides que não só encontra o máximo divisor comum (MDC) mas também os coeficientes de Bézout, ou seja, $\alpha\ ,\beta \in \mathbb{Z}$ tais que $\alpha a + \beta b = mdc(a,b) $.
Por isso ele é usado em criptografia para verificar se dois números são coprimos e encontrar coeficientes $\alpha$ e $\beta$ que satisfazem $\alpha a + \beta b = 1$

Ele funciona de forma iterativa e se baseia na fórmula do algoritmo de Euclides clássico incorporando $\alpha$ e $\beta$: 

$$ mdc(a,b) = mdc(b, a \mod (b)).$$

1. Se b=0, então:

$$mdc(a,b) = a, \alpha =1, \beta = 0$$

2. Caso contrário:
   - Use a fórmula $\text{MDC}(a, b) = \text{MDC}(b, a \mod b)$.
   
   - Resolva recursivamente $ b \cdot \alpha_1 + (a \mod b) \cdot \beta_1 = \text{MDC}(b, a \mod b) $.

   - Expanda $ a \mod b $ usando $ a \mod b = a - \lfloor a / b \rfloor \cdot b $:
     $$     a \cdot \beta_1 + b \cdot (\alpha_1 - \lfloor a / b \rfloor \cdot \beta_1) = \text{MDC}(a, b)     $$


   - Os novos coeficientes são:
     $$\alpha= \beta_1, \quad \alpha = \alpha_1 - \lfloor a / b \rfloor \cdot \beta_1    $$


In [48]:
import random
# from Crypto.Util.number import getPrime, inverse, GCD
from sympy import isprime
from math import gcd

In [49]:
class RSA:
    def __init__(self, bit_length=1024, generate_primes = True, first_prime = None, second_prime = None):
        self.bit_length = bit_length
        self.public_key = None
        self.private_key = None
        self.n = None
        self.generate_primes = generate_primes
        self.first_prime = first_prime
        self.second_prime = second_prime


    def extended_euclid(a, b):

        """
        Implementação do algoritmo estendido de Euclides do zero.
        
        Calcula o máximo divisor comum (MDC) entre a e b e os coeficientes de Bézout.
        
        Args:
            a (int): Primeiro número.
            b (int): Segundo número.
        
        Returns:
            tuple: (mdc, x, y), onde:
                mdc -> Máximo divisor comum entre a e b.
                x, y -> Coeficientes de Bézout tais que a * x + b * y = mdc.
        """
        x0, y0, x1, y1 = 1, 0, 0, 1  # Inicialização dos coeficientes de Bézout
        while b != 0:
            q = a // b  # Quociente da divisão inteira
            a, b = b, a % b  # Atualiza a e b usando o algoritmo de Euclides
            x0, x1 = x1, x0 - q * x1  # Atualiza os coeficientes x
            y0, y1 = y1, y0 - q * y1  # Atualiza os coeficientes y
        return a # a é o MDC, x0 e y0 são os coeficientes de Bézout
    
    def generate_prime(self):
        """Gera um número primo aleatório com o tamanho de bits especificado."""
        while True:
            num = random.getrandbits(self.bit_length // 2) # 2 números gerados, // significa div inteira
            if isprime(num): # verifica se os numeros são primos
                return num
    
    def generate_keys(self):
        """Gera as chaves pública e privada."""
        p=0
        q=0
        if self.generate_primes == True:  # Certifique-se de verificar o nome correto do atributo
            p = self.generate_prime()
            q = self.generate_prime()
        else:
            if self.first_prime is None or self.second_prime is None:
                raise ValueError("Você deve fornecer os dois números primos se 'generate_primes' for False.")
            p = self.first_prime
            q = self.second_prime
        
        self.n = p * q #parte 1 da chave pública
        phi = (p - 1) * (q - 1) # 
        
        # Escolha um 'e' tal que 1 < e < phi e gcd(e, phi) == 1
        # e = 65537  # Valor padrão usado na prática (é primo)
        e = 97
        if phi<=e:
            raise ValueError("Você precisa que e seja menor que phi. \n" +
                            "Por favor, selecione números primos maiores ou " +
                            "escolha uma chave e menor.")

        if RSA.extended_euclid(e, phi) != 1: #função que verifica o MDC entre dois números
            raise ValueError("e não é coprimo com φ(n), tente gerar os primos novamente.")
        
        # Calcula o 'd' (chave privada)
        d = pow(e, -1, phi)
        
        self.public_key = (e, self.n)
        self.private_key = (d, self.n)
    
    def encrypt(self, plaintext):
        """Criptografa uma mensagem, dividindo-a em blocos se necessário."""
        e, n = self.public_key
        max_block_size = (n.bit_length() - 1) // 8  # Calcula o tamanho máximo do bloco em bytes

        # Divide a mensagem em blocos menores que o tamanho máximo
        plaintext_bytes = plaintext.encode('utf-8')
        blocks = [plaintext_bytes[i:i + max_block_size] for i in range(0, len(plaintext_bytes), max_block_size)]

        # Criptografa cada bloco individualmente
        ciphertext_blocks = []
        for block in blocks:
            plaintext_int = int.from_bytes(block, byteorder='big')  # Converte o bloco para inteiro
            ciphertext = pow(plaintext_int, e, n)  # Criptografa o bloco
            ciphertext_blocks.append(ciphertext)

        return ciphertext_blocks  # Retorna a lista de blocos criptografados

    
    def decrypt(self, ciphertext_blocks):
        """
        Descriptografa uma lista de blocos criptografados.
        
        Args:
            ciphertext_blocks (list): Lista de blocos criptografados como inteiros.
        
        Returns:
            str: Mensagem descriptografada como string.
        """
        d, n = self.private_key

        # Descriptografa cada bloco
        plaintext_bytes = b""
        for block in ciphertext_blocks:
            plaintext_int = pow(block, d, n)  # Descriptografa o bloco
            block_bytes = plaintext_int.to_bytes((plaintext_int.bit_length() + 7) // 8, byteorder='big')
            plaintext_bytes += block_bytes  # Junta os bytes de cada bloco

        return plaintext_bytes.decode('utf-8')  # Converte para string

    
    def get_public_key(self):
        """Retorna a chave pública."""
        return self.public_key
    
    def get_private_key(self):
        """Retorna a chave privada."""
        return self.private_key

In [50]:

# Demonstração de uso
#Por favor não substitua os primos abaixo do exemplo (23,29)
if __name__ == "__main__":
    rsa = RSA(generate_primes=False, first_prime=23, second_prime=29)
    rsa.generate_keys()
    
    print("Chave pública:", rsa.get_public_key())
    print("Chave privada:", rsa.get_private_key())
    
    # mensagem = "Hello, RSA!"
    mensagem = "Mamadeira é muito bom, queria ter uma pra mamar todos os dias antes de ir dormir."
    print("\nMensagem original:", mensagem)
    
    criptografada = rsa.encrypt(mensagem)
    print("Mensagem criptografada:", criptografada)
    
    descriptografada = rsa.decrypt(criptografada)
    print("Mensagem descriptografada:", descriptografada)

Chave pública: (97, 667)
Chave privada: (489, 667)

Mensagem original: Mamadeira é muito bom, queria ter uma pra mamar todos os dias antes de ir dormir.
Mensagem criptografada: [32, 287, 352, 287, 9, 2, 95, 160, 287, 48, 272, 400, 48, 352, 581, 95, 116, 516, 48, 108, 516, 352, 201, 48, 155, 581, 2, 160, 95, 287, 48, 116, 2, 160, 48, 581, 352, 287, 48, 442, 160, 287, 48, 352, 287, 352, 287, 160, 48, 116, 516, 9, 516, 115, 48, 516, 115, 48, 9, 95, 287, 115, 48, 287, 633, 116, 2, 115, 48, 9, 2, 48, 95, 160, 48, 9, 516, 160, 352, 95, 160, 46]
Mensagem descriptografada: Mamadeira é muito bom, queria ter uma pra mamar todos os dias antes de ir dormir.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# Inicializar RSA e gerar chaves
rsa = RSA(bit_length=512)
rsa.generate_keys()

# Configurar o Selenium
driver = webdriver.Chrome()  # Certifique-se de ter o ChromeDriver instalado e configurado
driver.get("https://web.whatsapp.com/")

# Aguarde o QR Code ser escaneado
print("Escaneie o QR Code para acessar o WhatsApp Web")
time.sleep(25)  # Ajuste conforme necessário


# Encontre o contato e abra o chat

contatos = ['Wendell FGV', 'juliana fgv']
# contato = "juliana fgv"

for contato in contatos:
    print(f"Enviando para o contato {contato}")
    # Criptografar a mensagem
    mensagem = f"obrigado por interagir com a gente {contato}! ;)"
    criptografada = rsa.encrypt(mensagem)

    # Converter os blocos criptografados para string legível
    mensagem_enviar = " ".join(map(str, criptografada))

    try:
        contato_element = driver.find_element(By.XPATH, f"//span[@title='{contato}']")
        contato_element.click()
    except Exception as e:
        print("Erro ao encontrar o contato. Verifique o nome e tente novamente.")
        driver.quit()
        raise e

    time.sleep(5)

    try:
        # Localizar e clicar no contato
        contato_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//span[@title='{contato}']"))
        )
        contato_element.click()
    except Exception as e:
        print(f"Erro ao encontrar o contato '{contato}'. Verifique o nome e tente novamente.")
        continue  # Pula para o próximo contato

    time.sleep(2)  # Aguardar a interface carregar

    # Envie a mensagem no campo de mensagens do contato
    try:
        # Localizar o campo de mensagem
        caixa_mensagem = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//footer//div[@contenteditable='true']"))
        )
        caixa_mensagem.click()

        # Enviar a primeira mensagem
        caixa_mensagem.send_keys("Mensagem criptografada:")
        caixa_mensagem.send_keys(Keys.ENTER)

        time.sleep(1)  # Pequena pausa para evitar problemas de sincronização

        # Enviar a mensagem criptografada
        caixa_mensagem.send_keys(f"\"{mensagem_enviar}\"")
        caixa_mensagem.send_keys(Keys.ENTER)
    except Exception as e:
        print(f"Erro ao enviar a mensagem para '{contato}'. Verifique o seletor do campo de mensagem.")
        continue  # Pula para o próximo contato

    time.sleep(2)  # Tempo entre mensagens para evitar problemas com o WhatsApp Web
# try:
#     caixa_mensagem = driver.find_element(By.XPATH, "//footer//div[@contenteditable='true']")  # Localização do campo de mensagem
#     caixa_mensagem.click()
#     caixa_mensagem.send_keys(f"Chave pública: {rsa.public_key}")
#     caixa_mensagem.send_keys(Keys.ENTER)
# except Exception as e:
#     print("Erro ao enviar a mensagem. Verifique o seletor do campo de mensagem.")
#     driver.quit()
#     raise e
# Fechar o navegador
time.sleep(5)
driver.quit()




Escaneie o QR Code para acessar o WhatsApp Web
Enviando para o contato juliana fgv
Enviando para o contato Wendell FGV
Erro ao encontrar o contato. Verifique o nome e tente novamente.


ElementClickInterceptedException: Message: element click intercepted: Element <span dir="auto" title="..." aria-label="" class="x1iyjqo2 x6ikm8r x10wlt62 x1n2onr6 xlyipyv xuxw1ft x1rg5ohu _ao3e" style="min-height: 0px;">Wendell FGV</span> is not clickable at point (188, 590). Other element would receive the click: <div class="x1c4vz4f xs83m0k xdl72j9 x1g77sc7 xozqiw3 x1oa3qoh x12fk4p8 xeuugli x2lwn1j xaw8158 x1q0g3np x6s0dn4 x182zwpg x3nfvp2">...</div>
  (Session info: chrome=124.0.6367.119)
Stacktrace:
	GetHandleVerifier [0x00007FF7B35F1522+60802]
	(No symbol) [0x00007FF7B356AC22]
	(No symbol) [0x00007FF7B3427CE4]
	(No symbol) [0x00007FF7B347E877]
	(No symbol) [0x00007FF7B347C344]
	(No symbol) [0x00007FF7B347980A]
	(No symbol) [0x00007FF7B34789E2]
	(No symbol) [0x00007FF7B346AE8B]
	(No symbol) [0x00007FF7B349AB7A]
	(No symbol) [0x00007FF7B346A7C6]
	(No symbol) [0x00007FF7B349AD90]
	(No symbol) [0x00007FF7B34BA224]
	(No symbol) [0x00007FF7B349A923]
	(No symbol) [0x00007FF7B3468FEC]
	(N

In [ ]:
print(f"Chave privada: {rsa.private_key}")
private_key = rsa.private_key

Chave privada: (2873562672119053333235873444239542578673150819143991694110402830928349728296577964897862099211812796092498951005997391438765406822601872936475084392153, 21441198399657551794144594160864279240868894573612861102208390353849994126587413561127081875118696313082175409045648800899265756441180310040467539971483)


In [ ]:
#private_key = "digite aqui sua chave privada (com parêntesis)"
#mensagem_criptografada = "digite aqui a mensagem a ser descriptografada "com aspas" "
mensagem_criptografada = "3001935664968425618048279896013018752241251386471245344764340133065513947980121691417049659559262573821726806229237868971211771930485669834445010334158 8107503201061340885677330499551110185884391208833424854225012289479345882235173454791556486354804587252340436314607326560958170475853280669355489575025"

# Converter a mensagem criptografada para uma lista de inteiros
ciphertext_blocks = list(map(int, mensagem_criptografada.split()))

# Descriptografar usando a chave privada
def decrypt(private_key, ciphertext_blocks):
    d, n = private_key
    plaintext_bytes = b""
    for block in ciphertext_blocks:
        plaintext_int = pow(block, d, n)  # Descriptografa o bloco
        block_bytes = plaintext_int.to_bytes((plaintext_int.bit_length() + 7) // 8, byteorder='big')
        plaintext_bytes += block_bytes
    return plaintext_bytes.decode('utf-8')

# Executar a descriptografia
mensagem_original = decrypt(private_key, ciphertext_blocks)
print("Mensagem descriptografada:", mensagem_original)


Mensagem descriptografada: Esta é uma mensagem criptografada pelo RSA! Atnciosamente, Daniel Jacob
